In [1]:
from setup_mnist import MNIST
import helper
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn import svm
from cvxopt import matrix, solvers
from itertools import product
import sys
%matplotlib inline

Using TensorFlow backend.


In [2]:
data = MNIST()

In [3]:
X_train = data.train_data.reshape(-1, 28*28)
Y_train = np.argmax(data.train_labels, axis=1)

In [4]:
X_test = data.test_data.reshape(-1, 28*28)
Y_test = np.argmax(data.test_labels, axis=1)

In [5]:
np.histogram(Y_train)

(array([5444, 6179, 5470, 5638, 5307, 4987, 5417, 5715, 5389, 5454]),
 array([ 0. ,  0.9,  1.8,  2.7,  3.6,  4.5,  5.4,  6.3,  7.2,  8.1,  9. ]))

In [6]:
def getMax(lst, target):
    """
    returns maximum of the list (ix, elt); omits target entry
    """
    n1 = (-sys.maxint, None)
    for ix, elt in enumerate(lst):
        if ix == target:
            continue
        elif elt > n1[1]:
            n1 = (ix, elt)
    return n1


In [21]:
def get_second(lst):import numpy as np
from sklearn import svm


class LinearBinaryClassifier(object):
    """
    Class for Linear Binary Classifiers

    weights: np array of shape (dim, 1)
    bias: scalar
    """

    def __init__(self, weights, bias):
        self.dim = weights.shape[0]
        self.weights = weights
        self.bias = bias

    def predict(self, X):
        """
        X: np array of shape (num_points, dim)

        returns: a vector of shape (num_points,) with predicted labels for each point
        """
        return np.sign(np.matmul(X, self.weights) + self.bias).T[0]

    def distance(self, X):
        """
        Computes the signed distance from a point to the decision boundary (hyperplane)

        returns: a vector of shape (num_points,) with the correspoding distances
        """
        return abs((np.matmul(X, self.weights) + self.bias) / np.linalg.norm(self.weights)).T[0]

    def evaluate(self, X, Y):
        """
        returns accuracy
        """
        return np.mean(np.equal(self.predict(X), Y))

    def gradient(self, X, Y):
        """
        returns gradient
        """
        if not hasattr(Y, "__len__"):  # make it robust to single items
            X = X.reshape(1, self.dim)
            Y = np.array([Y])

        return np.array([Y[i] * self.weights.reshape(-1, ) if self.predict(X[i]) == Y[i]
                         else np.zeros(self.dim) for i in xrange(len(X))])

    def rhinge_loss(self, X, Y):
        """
        returns average reverse hinge loss of classifier on X, Y

        defined as max{0, y(<w,x> + b)}
        """
        if not hasattr(Y, "__len__"):  # make it robust to single items
            X = X.reshape(1, self.dim)
            Y = np.array([Y])

        res = np.maximum(0, Y.reshape(-1, 1) * (np.matmul(X, self.weights) + self.bias))
        return np.mean(res.reshape(-1, ))


def trainLBC(X, Y):
    model = svm.SVC(kernel="linear")
    model.fit(X, Y)
    return LinearBinaryClassifier(model.coef_.T, model.intercept_)


class LinearOneVsAllClassifier(object):
    """
    Class for Lin Binary Classifiers

    weights: np array of shape (num_classes, dim)
    bias: scalar
    """

    def __init__(self, num_classes, weights, bias):
        self.dim = weights.shape[1]
        self.weights = weights
        self.bias = bias
        self.num_classes = num_classes

    def predict(self, X):
        """
        X: np array of shape (num_points, dim)

        returns: a vector of shape (num_points,) with predicted labels for each point
        """
        return np.argmax(np.matmul(X, self.weights.T) + self.bias, axis=1)

    def distance(self, X):
        """
        Computes the signed distance from a point to the decision boundary (hyperplane)

        returns: a vector of shape (num_points,) with the correspoding distances
        """
        n = X.shape[0]
        Y = self.predict(X)

        distances = []
        for i in xrange(n):
            label_options = range(10)
            del label_options[Y[i]]
            dists = []
            for j in label_options:
                v = tryRegionOneVsAll([self], [j], X[i])
                dists.append(np.linalg.norm(v))
            distances.append(min(dists))
        return distances

    def evaluate(self, X, Y):
        """
        returns accuracy
        """
        return np.mean(np.equal(self.predict(X), Y))

    def gradient(self, X, targets):
        """
        returns gradient
        """
        preds = np.matmul(X, self.weights.T) + self.bias
        n = X.shape[0]

        gradient = []

        for i in xrange(n):
            target = targets[i]
            others = range(10)
            del others[target]

            if np.argmax(preds[i]) == target:
                res = np.zeros(self.dim)
            else:
                max_ix = getMax(preds[i], target)[0]
                w_max = self.weights[max_ix]
                w_target = self.weights[target]
                res = w_max - w_target
            gradient.append(res)
        return np.array(gradient)

    def rhinge_loss(self, X, targets):
        preds = np.matmul(X, self.weights.T) + self.bias
        res = []
        for i in xrange(len(X)):
            target = targets[i]
            if np.argmax(preds) != target:
                max_ix, max_val = getMax(preds[i], target)
                loss = max_val - preds[i][target]
            else:
                loss = 0
            res.append(loss)
        return res
    
    def untargeted_loss(self, X, Y):
        preds = np.matmul(X, self.weights.T) + self.bias
        n = X.shape[0]
        loss = []

        for i in xrange(n):
            y = Y[i]
            others = range(10)
            del others[y]
            if np.argmax(preds[i]) != y:
                res = 0
            else:
                max_val = getMax(preds[i], y)[1]
                y_val = preds[i][y]
                res = y_val - max_val
            loss.append(res)
        return np.array(loss)

    
    def gradient_untargeted(self, X, Y):
        """
        returns gradient
        """
        preds = np.matmul(X, self.weights.T) + self.bias
        n = len(X)
        gradient = []

        for i in xrange(n):
            y = Y[i]
            others = range(10)
            del others[y]
            if np.argmax(preds[i]) != y:
                res = np.zeros(self.dim)
            else:
                max_ix = getMax(preds[i], y)[0]
                w_max = self.weights[max_ix]
                w_y = self.weights[y]
                res = w_y - w_max
            gradient.append(res)
        return np.array(gradient)


def trainLMC(X, Y, method):
    if method == "one-vs-all":
        model = svm.LinearSVC(loss='hinge')
    else: # "all-pairs"
        model= svm.SVC(kernel="linear")
    model.fit(X, Y)
    if method == "one-vs-all":
        res = LinearOneVsAllClassifier(10, model.coef_, model.intercept_)
    return res

    n1 = (-sys.maxint, None)
    n2 = (-sys.maxint, None)
    for ix, elt in enumerate(lst):
        if elt > n1[0]:
            n2 = n1
            n1 = (elt, ix)
        elif elt > n2[0]:
            n2 = (elt, ix) 
    return n2

def getMax(lst, target):
    n1 = (-sys.maxint, None)
    for ix, elt in enumerate(lst):
        if ix == target:
            continue
        elif elt > n1[1]:
            n1 = (ix, elt)
    return n1

In [9]:
getMax([1,-1,3], 2)

(0, 1)

In [10]:
def trainLMC(X, Y, method):
    if method == "one-vs-all":
        model = svm.LinearSVC(loss='hinge')
    else: # "all-pairs"
        model= svm.SVC(kernel="linear")
    model.fit(X, Y)
    if method == "one-vs-all":
        res = LinearOneVsAllClassifier(10, model.coef_, model.intercept_)
    return res

In [22]:
# train the classifiers
n = 3
train_size = 3000 #len(X_train) / n

classifiers = []

for i in xrange(n):
    start = train_size * i
    end = start + train_size
    lmc = trainLMC(X_train[start:end], Y_train[start:end], "one-vs-all")
    print i, start, end, lmc.evaluate(X_test, Y_test)
    classifiers.append(lmc)

0 0 3000 0.8629
1 3000 6000 0.8615
2 6000 9000 0.8635


In [ ]:
d = [model.distance(X_test[:10]) for model in classifiers]

In [ ]:
x = np.array([X_test[0]])
y = [Y_test[0]]
print y

In [ ]:
print classifiers[0].evaluate(x,y)
classifiers[0].gradient(x, [4])

In [ ]:
classifiers[0].weights[0].shape

In [ ]:
classifiers[0].bias[0]

In [12]:
def tryRegionOneVsAll(models, labels, x, delta=1e-10):
    P = matrix(np.identity(x.shape[0]))
    q = matrix(np.zeros(x.shape[0]))
    h = []
    G = []
    num_models = len(models)
    for i in xrange(num_models):
        others = range(10)
        target = labels[i]
        del others[target]
        target_w, target_b = models[i].weights[target], models[i].bias[target]
        for j in others:
            other_w, other_b = models[i].weights[j], models[i].bias[j]
            ineq_val = np.dot(target_w - other_w, x) + target_b - other_b - delta
            h.append(ineq_val)
            G.append(other_w - target_w)   
    h = matrix(h)
    G = matrix(np.array(G))
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h)
    if sol['status'] == 'optimal':
        v = np.array(sol['x']).reshape(-1,)
        perturbed_x = np.array(x + v).reshape(1, -1)
        is_desired_label = [models[i].predict(perturbed_x)[0] == labels[i] for i in xrange(num_models)]
        if sum(is_desired_label) == num_models:
            return v
        else:
            return tryRegionOneVsAll(models, labels, x, delta * 1.5)
    else:
        return None

In [ ]:
x = X_test[0]
y = Y_test[0]
print y

In [ ]:
v = tryRegionOneVsAll(classifiers, [5, 4, 9, 2, 1], x)
print np.linalg.norm(v)
[c.predict((x + v).reshape(1, -1)) for c in classifiers]

In [13]:
def distributionalOracleOneVsAll(distribution, models, x, y, alpha):
    
    candidates = []
    # we should only take into consideration models that we could feasibly trick

    num_models = len(models)
    
    labels_values = []
    for labels in product(range(10), repeat=num_models): # iterate over all possible regions
        is_misclassified = (np.array(labels) != y).astype(np.float32)
        value = np.dot(is_misclassified, distribution)
        labels_values.append((labels, value))
    
    values = sorted(set([value for label, value in labels_values]), reverse=True)
    
    for curr_value in values:
#         print "Curr Value", curr_value
        feasible_candidates = []
        for labels in [labels for labels, val in labels_values if val == curr_value]:
#             print labels
            v = tryRegionOneVsAll(models, labels, x)
            if v is not None:
                norm = np.linalg.norm(v)
                if norm <= alpha:
                    feasible_candidates.append((v, norm))
        # amongst those with the max value, return the one with the minimum norm
        if feasible_candidates:
            # break out of the loop since we have already found the optimal answer
#             print "curr_value ", curr_value
            return min(feasible_candidates, key=lambda x: x[1])[0]
    return np.zeros(x.shape[0]) # we can't trick anything

In [ ]:
s = time.time()
v = distributionalOracleOneVsAll([1, .2, .6, .4, .5], classifiers, x, y, 1)
print time.time() - s
print np.linalg.norm(v)
[c.predict((x + v).reshape(1,-1)) for c in classifiers]

In [ ]:
class_means = []
for i in xrange(10):
    elts = np.array([x for (x,y) in zip(X_train, Y_train) if y == i])
    mean = np.mean(elts, axis=0)
    class_means.append(mean)
class_means = np.array(class_means)

In [ ]:
mean_diffs = []
for i in xrange(10):
    diffs = []
    for j in xrange(10):
        diffs.append(np.linalg.norm(class_means[i] - class_means[j]))
    mean_diffs.append(diffs)

In [ ]:
mean_diffs = np.array(mean_diffs)

In [ ]:
MEAN_DICT = dict(zip(range(10), [getMax(mean_diffs[i] * -1, i)[0] for i in range(10)]))

In [ ]:
np.save("greedy_dict.npy", np.array(MEAN_DICT.items()))

In [ ]:
def coordinateAscentMulti(distribution, models, x, y, alpha, greedy=False):
    
    num_models = len(models)
    
    sol = np.zeros(x.shape)

    labels = [y] * num_models # initialize to the original point, of length feasible_models
    label_options = range(10)
    del label_options[y]
    
    model_options = dict(zip(range(num_models), distribution))
    for i in xrange(num_models):
        if greedy:
            coord = max(model_options, key=model_options.get)
            labels[coord] = greedy[y]
        else:
            coord = np.random.choice(model_options.keys())
            labels[coord] = np.random.choice(label_options)

        del model_options[coord]    
       
        v = tryRegionOneVsAll(models, labels, x)
        valid_sol = False
        if v is not None:
            norm = np.linalg.norm(v)
            if norm <= alpha:
                valid_sol = True
                sol = v
        if not valid_sol:
            break 
    return sol

In [ ]:
v = coordinateAscentMulti([.2, .2, .6], classifiers, x, y, 1)
print np.linalg.norm(v)
[c.predict((x + v).reshape(1,-1)) for c in classifiers]

In [ ]:
def gradientDescentTargeted(distribution, models, x, target, alpha, learning_rate=.001, T=3000, early_stop=5):
    v = np.zeros(len(x))
    best_sol = (sys.maxint, v)
    loss_queue = []
    for i in xrange(T):
        gradient = sum([-1 * p * model.gradient(np.array([x + v]), [target]) for p, model in zip(distribution, models)])[0]
       
        v += learning_rate * gradient
        norm  = np.linalg.norm(v)
        if norm >= alpha:
            v = v / norm * alpha
            
        loss = np.dot(distribution, [model.rhinge_loss([x + v], [target])[0] for model in models])
        
        loss_queue = [loss] + loss_queue
        if i >= early_stop:
            del loss_queue[-1]
#             print "Len ", len(loss_queue)
            val = loss_queue[-1]
            if sum([val == q_val for q_val in loss_queue]) == early_stop:
                break
    
        if loss < best_sol[0]:
            best_sol = (loss, v)
        
        if loss == 0:
#             print "FOUND IT"
            break
            
#         print i, loss, [model.predict(curr_sol) for model in models], np.linalg.norm(v)
        
    return best_sol

In [ ]:
def gradientDescentUntargeted(distribution, models, x, y, alpha):
    targets = range(10)
    del targets[y]
    noise_options = []
    for target in targets:
#         print "Target ", target
        sol = gradientDescentTargeted(distribution, models, x, target, alpha)
        noise_options.append(sol)
        if sol[0] == 0:
#             print "BREAK"
#             print [model.predict((x + sol[1]).reshape(1,-1)) for model in models]
            return sol[1]
    return min(noise_options, key=lambda x:x[0])[1]

In [30]:
def gradientDescentNonConvex(distribution, models, x, y, alpha, learning_rate=.001, T=3000, early_stop=5):
    v = np.zeros(len(x))
    best_sol = (sys.maxint, v)
    loss_queue = []
    for i in xrange(T):
        gradient = sum([-1 * p * model.gradient_untargeted(np.array([x + v]), [y]) 
                        for p, model in zip(distribution, models)])[0]
        v += learning_rate * gradient
        norm  = np.linalg.norm(v)
        if norm >= alpha:
            v = v / norm * alpha
            
        loss = np.dot(distribution, [model.untargeted_loss(np.array([x + v]), [y])[0] for model in models])
        
        loss_queue = [loss] + loss_queue
        if i >= early_stop:
            del loss_queue[-1]
            val = loss_queue[-1]
            if sum([val == q_val for q_val in loss_queue]) == early_stop:
                break
    
        if loss < best_sol[0]:
            best_sol = (loss, v)
        
        if loss == 0:
            print "FOUND IT"
            break
            
        print i, loss, [model.predict(np.array([x + v])) for model in models], np.linalg.norm(v)
        
    return best_sol[1]

In [34]:
x = X_test[5]
y = Y_test[5]
print y

1


In [35]:
s = time.time()
v = gradientDescentNonConvex([1, 1, 1], classifiers, x, y, 1)
print "time ", time.time() - s 
print "norm ", np.linalg.norm(v)
print [c.predict((x + v).reshape(1,-1)) for c in classifiers]

0 10.7266851047 [array([1]), array([1]), array([1])] 0.0188645066253
1 10.3708154945 [array([1]), array([1]), array([1])] 0.0377290132506
2 10.0149458843 [array([1]), array([1]), array([1])] 0.0565935198758
3 9.65907627408 [array([1]), array([1]), array([1])] 0.0754580265011
4 9.30320666387 [array([1]), array([1]), array([1])] 0.0943225331264
5 8.94733705365 [array([1]), array([1]), array([1])] 0.113187039752
6 8.59146744343 [array([1]), array([1]), array([1])] 0.132051546377
7 8.23559783322 [array([1]), array([1]), array([1])] 0.150916053002
8 7.879728223 [array([1]), array([1]), array([1])] 0.169780559627
9 7.52385861279 [array([1]), array([1]), array([1])] 0.188645066253
10 7.16798900257 [array([1]), array([1]), array([1])] 0.207509572878
11 6.81211939236 [array([1]), array([1]), array([1])] 0.226374079503
12 6.45624978214 [array([1]), array([1]), array([1])] 0.245238586129
13 6.10038017193 [array([1]), array([1]), array([1])] 0.264103092754
14 5.74451056171 [array([1]), array([1]),

In [ ]:
from helper import generate_data

In [ ]:
X_exp, Y_exp = generate_data(100, X_test, Y_test, classifiers)

In [ ]:
d = np.array([model.distance(X_exp) for model in classifiers])

In [ ]:
np.max(d), np.min(d), np.mean(d)

In [ ]:
from mwu import runMWU

In [ ]:
from functools import partial
greedyCoordinateAscent = partial(coordinateAscentMulti, greedy=MEAN_DICT)
randomCoordinateAscent = partial(coordinateAscentMulti, greedy=False)

In [ ]:
def findNoiseBoundsMulti(models, X, Y):
    max_bounds = []
    num_models = len(models)
    for i in xrange(len(X)):
        max_v = distributionalOracleOneVsAll([1] * num_models, models, X[i], Y[i], sys.maxint)
        max_bounds.append(np.linalg.norm(max_v))
    min_bounds = np.array([model.distance(X) for model in models]).T
    min_bounds = np.mean(min_bounds, axis=1)
    return min_bounds, max_bounds

In [ ]:
res = findNoiseBoundsMulti(classifiers, X_exp[:3], Y_exp[:3])

In [ ]:
res

In [ ]:
import os
os.mkdir("trash")

In [ ]:
def evaluateCosts(models, V, X, Y):
    return np.array([1 - model.evaluate(X + V, Y) for model in models])


def adversary(distribution, models, X, Y, alpha, noiseFunc):
    return np.array([noiseFunc(distribution, models, x, y, alpha) for x, y in zip(X,Y)])


def runMWU(models, T, X, Y, alpha, noiseFunc, exp_dir, epsilon=None):
    num_models = len(models)

    if epsilon is None:
        delta = np.sqrt(4 * np.log(num_models) / float(T))
        epsilon = delta / 2.0
    else:
        delta = 2.0 * epsilon

    print("\nRunning MWU for {} Iterations with Epsilon {}\n".format(T, epsilon))

    print("Guaranteed to be within {} of the minimax value \n".format(delta))

    loss_history = []
    costs = []
    max_acc_history = []
    v = []
    w = []
    action_loss = []

    w.append(np.ones(num_models) / num_models)

    for t in xrange(T):
        print("Iteration {}\n".format(t))

#         if t % (T * .10) == 0 and t > 0:
#             np.save(exp_dir + "/" + "weights_{}.npy".format(t), w)
#             np.save(exp_dir + "/" + "noise_{}.npy".format(t), v)
#             np.save(exp_dir + "/" + "loss_history_{}.npy".format(t), loss_history)
#             np.save(exp_dir + "/" + "max_acc_history_{}.npy".format(t), max_acc_history)
#             np.save(exp_dir + "/" + "action_loss_{}.npy".format(t), action_loss)

        start_time = time.time()

        v_t = adversary(w[t], models, X, Y, alpha, noiseFunc)
        v.append(v_t)

        cost_t = evaluateCosts(models, v_t, X, Y)
        costs.append(cost_t)

        avg_acc = np.mean((1 - np.array(costs)), axis=0)
        max_acc = max(avg_acc)
        max_acc_history.append(max_acc)

        loss = np.dot(w[t], cost_t)
        individual = [w[t][j] * cost_t[j] for j in xrange(num_models)]

        print("Weights {} Sum of Weights {}".format(w[t], sum(w[t])))
        print("Maximum (Average) Accuracy of Classifier {}".format(max_acc))
        print("Cost (Before Noise) {}".format(np.array([1 - model.evaluate(X, Y) for model in models])))
        print("Cost (After Noise), {}".format(cost_t))
        print("Loss {} Loss Per Action {}".format(loss, individual))

        loss_history.append(loss)
        action_loss.append(individual)

        new_w = np.copy(w[t])

        # penalize experts
        for i in xrange(num_models):
            new_w[i] *= (1.0 - epsilon) ** cost_t[i]

        # renormalize weights
        w_sum = new_w.sum()
        for i in xrange(num_models - 1):
            new_w[i] = new_w[i] / w_sum
        new_w[-1] = 1.0 - new_w[:-1].sum()

        w.append(new_w)

        print("time spent {}\n".format(time.time() - start_time))
    print("finished running MWU ")
    return w, v, loss_history, max_acc_history, action_loss


In [ ]:
res = runMWU(classifiers, 30, X_exp, Y_exp, .3, gradientDescentUntargeted, "trash")

In [ ]:
x = X_exp[0]
y = Y_exp[0]
print y

In [ ]:
v = gradientDescentMulti([.2,.2,.6], classifiers, x, 5, 1, learning_rate=.0001, T=4000)
print np.linalg.norm(v)
[c.predict((x + v).reshape(1,-1)) for c in classifiers]

In [ ]:
s.shape